# LUX Acadamy Data Science BootCamp Project , Dawit Tadesse, Ethiopia

In [ ]:
# if on google colab, install kaggle
!pip install kaggle

In [ ]:
# create the .kaggle directory so that you can configure kaggle using your kaggle.json api key information
!mkdir -p ~/.kaggle


In [ ]:
# copies the kaggle json file
# make sure that the kaggle.json file is uplloaded into colab or that it's inside the project
# you can use the upload button on the side menu in colab after clicking the file icon
!cp kaggle.json ~/.kaggle/

In [ ]:
# permissions configuration, 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#check directory 
!pwd

In [ ]:
#download the datasets
!kaggle datasets download -d antoreepjana/animals-detection-images-dataset

In [ ]:
#unzip the datasets
!unzip animals-detection-images-dataset.zip

In [ ]:
#list the classes in the test directory
import os
os.listdir("./test")

In [ ]:
# creae the paths so we can use them later
test_dir = './test'
train_dir = './train'

In [ ]:
# save the labes into variable so we can use them later and also check how many labes we have
# for the softmax classification
labels = os.listdir(test_dir)
print(len(labels))

In [ ]:
# check a random image to see if all is going well
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg



img = mpimg.imread('./test/Brown bear/037ce70dca241ef6.jpg')
plt.imshow(img)
plt.axis('Off')
plt.show()

In [ ]:
# let's preprocess the images using tensorflows image generator class
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Image augmentation so that we can get better results on our classification
training_data = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range=0.2,
                                height_shift_range=0.2,
                                  rotation_range=20,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')
# no augmentation on the test data
validation_data =  ImageDataGenerator(rescale = 1./255)

train_generator = training_data.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    class_mode='categorical',
                                                    batch_size=64)


valid_generator = validation_data.flow_from_directory(test_dir,
                                                    target_size=(224,224),
                                                    class_mode='categorical',
                                                    batch_size=64)

In [ ]:
import tensorflow as tf
# Lets create our model
# our models is a convolutional neural net
# with the following layers
# 64,64,128,128, Flatten, Dropout, 512, 80
# the image's are 224 by 224 with three channels
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 224x224 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(80, activation='softmax')
])

# Print the model summary
model.summary()

In [ ]:
# Set the training parameters, we are using adam optimizer and accuracy for metrics
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model, we let the model figure out the optimal steps 
# By the way, if running on colab, run it using GPU environment to make 
# the trianing faster, or else you will be waiting for a long time
history = model.fit(train_generator, validation_data = valid_generator, epochs=10)

In [ ]:
# Plot the model results to visulal see the result
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()


In [ ]:
# If your doing this on colab
# use the following code to check for an image
#from google.colab import files
#from tensorflow.keras.utils import load_img, img_to_array

#uploaded = files.upload()

#for fn in uploaded.keys():
 
  # predicting images
#  path = fn
#  img = load_img(path, target_size=(224, 224))
#  x = img_to_array(img)
#  x = x  * 1./255
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
    
#  classes = model.predict(images)
#  print(fn)
#  print(classes)

In [ ]:
from PIL import Image
import io
import base64

#This function assumes that the input data is a JSON object containing a 
#base64-encoded image under the "image" key from 
# the post request.  Can adjust the input format as needed.


def preprocess(data):
    # Decode the base64 image
    img_data = base64.b64decode(data["image"])
    img = Image.open(io.BytesIO(img_data))

    # Resize the image to the target size (224,224)
    img = img.resize((224, 224), Image.ANTIALIAS)

    # Convert the image to a NumPy array and apply the rescale factor
    image_array = np.array(img) / 255.0

    # Add a batch dimension and return the processed image
    return np.expand_dims(image_array, axis=0)


In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf

app = Flask(__name__)

# Load the saved TensorFlow model
#model = tf.keras.models.load_model("my_model.h5")

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    # Ensure the input data is in the correct format
    image_data = preprocess(data)
    
    # Make a prediction using the loaded model
    predictions = model.predict(image_data)
    
    # Convert the predictions to a JSON response
    response = jsonify(predictions.tolist())
    # return the raw results to the user making the requests
    return response

#if __name__ == '__main__':
    app.run(debug=True)



In [3]:
# End Note: you can also save the models instead and run the deployment on another .py file on your computer